
# Importing Packages and Dataset

In [1]:
# import packages for data cleaning and processing  
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

# import modules for preprocessing
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from mlxtend.evaluate import bias_variance_decomp

# pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [3]:
df = pd.read_csv('data/kc_house_data.csv')

In [4]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [6]:
df.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

# Feature Selection

In [7]:
# Extract sale date for later processing
df['sale_date'] = [x[:8] for x in df.date]
df.sale_date = df.sale_date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))
df.drop(columns='date', inplace=True)
df.drop(['id'], inplace=True, axis=1)

# Replace anomalous bedroom value and check values in column
df.replace({'bedrooms': {33: 3}}, inplace=True)
df.replace({'bathrooms': {0: 0.25}}, inplace=True)

# Create new feature to incorporate age at the time of sale
df['sale_age'] = df.sale_date.dt.year - df[['yr_built', 'yr_renovated']].max(axis=1)
df.replace({'sale_age': {-1: 0}}, inplace=True)

# Create new feature for age from year built
df['age'] = df.sale_date.dt.year - df.yr_built
df.replace({'age': {-1: 0}}, inplace=True)

# Create binary variables for whether there has been a renovation, the property has a bathroom, and has been viewed
df['renovated'] = df.yr_renovated.apply(lambda x: x if x==0 else 1)
df['basement'] = df.sqft_basement.apply(lambda x: x if x==0 else 1)
df['viewed'] = df.view.apply(lambda x: x if x==0 else 1)

# Drop original columms as well as the sale_date columns since it is in datetime format
df.drop(['yr_built', 'yr_renovated', 'sale_date', 'sqft_basement', 'view'], inplace=True, axis=1)

# Drop latitude and longitude as zip code has stronger correlation
df.drop(['lat', 'long'], inplace=True, axis=1)

In [8]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,zipcode,sqft_living15,sqft_lot15,sale_age,age,renovated,basement,viewed
0,221900.0,3,1.00,1180,5650,1.0,0,3,7,1180,98178,1340,5650,59,59,0,0,0
1,538000.0,3,2.25,2570,7242,2.0,0,3,7,2170,98125,1690,7639,23,63,1,1,0
2,180000.0,2,1.00,770,10000,1.0,0,3,6,770,98028,2720,8062,82,82,0,0,0
3,604000.0,4,3.00,1960,5000,1.0,0,5,7,1050,98136,1360,5000,49,49,0,1,0
4,510000.0,3,2.00,1680,8080,1.0,0,3,8,1680,98074,1800,7503,28,28,0,0,0


# Dummy Variables and Polynomial/Interaction Features

In [9]:
# Grab indices of columns for creating dummy variables and create dataframe with dummy variables
dum_feat = df[['bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'zipcode']]
dum_index = dum_feat.columns

# Create dummy variables then drop one of the dummy variables, as well as original categorical variable used in creating the dummy variables
df_dum = pd.get_dummies(data=dum_feat, columns=dum_index, drop_first=True, prefix=['bdr', 'bth', 'flr', 'cnd', 'grd', 'zip'])
df_dum.head()

,bdr_1,bdr_2,bdr_3,bdr_4,bdr_5,bdr_6,bdr_7,bdr_8,bdr_9,bdr_10,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Set target variable
y = df['price']

# Baseline Model

In [25]:
poly_feat_1 = df.drop(['price', 'bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'zipcode'], axis=1)
X = pd.concat([poly_feat_1, df_dum], axis=1)
X.head()

,sqft_living,sqft_lot,waterfront,sqft_above,sqft_living15,sqft_lot15,sale_age,age,renovated,basement,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,1180,5650,0,1180,1340,5650,59,59,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2570,7242,0,2170,1690,7639,23,63,1,1,...,0,0,0,0,0,0,0,0,0,0
2,770,10000,0,770,2720,8062,82,82,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1960,5000,0,1050,1360,5000,49,49,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1680,8080,0,1680,1800,7503,28,28,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True)

In [34]:
lr = LinearRegression().fit(X_train, y_train)
loss, bias, var = bias_variance_decomp(lr, X_train.values, y_train.values, X_test.values, y_test.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss)
print('Average bias: %.3e' % bias)
print('Average variance: %.3e' % var)
y_pred_train_lr = lr.predict(X_train)
y_pred_test_lr = lr.predict(X_test)
train_test_metrics(y_train, y_test, y_pred_train_lr, y_pred_test_lr)


Average expected loss: 3.245e+10
Average bias: 3.078e+10
Average variance: 1.676e+09
Training R^2 Score:  0.8328
Training RMSE: 147801
Testing R^2 Score:  0.8147
Testing RMSE: 167382


# Degree-2 Polynomial Features Model

In [39]:
# Grab columns for polynominal and interaction features from the original dataframe without dummy variables
poly_feat_2 = df.drop(['price', 'bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'zipcode'], axis=1)
# Use PolynomialFeatures to create binomial and interaction features
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly_data_2 = poly_2.fit_transform(poly_feat_2)
poly_columns_2 = poly_2.get_feature_names(poly_feat_2.columns)
df_poly_2 = pd.DataFrame(poly_data_2, columns=poly_columns_2)
# Concatenating two dataframes together for input into linear regression model
X_poly_2 = pd.concat([df_poly_2, df_dum], axis=1)
X_poly_2.head()

,sqft_living,sqft_lot,waterfront,sqft_above,sqft_living15,sqft_lot15,sale_age,age,renovated,basement,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,1180.0,5650.0,0.0,1180.0,1340.0,5650.0,59.0,59.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1,2570.0,7242.0,0.0,2170.0,1690.0,7639.0,23.0,63.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,770.0,10000.0,0.0,770.0,2720.0,8062.0,82.0,82.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1960.0,5000.0,0.0,1050.0,1360.0,5000.0,49.0,49.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,1680.0,8080.0,0.0,1680.0,1800.0,7503.0,28.0,28.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_poly_2, y, random_state=42, test_size=0.2, shuffle=True)
lr_2 = LinearRegression().fit(X_train_2, y_train_2)
loss_2, bias_2, var_2 = bias_variance_decomp(lr_2, X_train_2.values, y_train_2.values, X_test_2.values, y_test_2.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_2)
print('Average bias: %.3e' % bias_2)
print('Average variance: %.3e' % var_2)
y_pred_train_lr_2 = lr_2.predict(X_train_2)
y_pred_test_lr_2 = lr_2.predict(X_test_2)
train_test_metrics(y_train_2, y_test_2, y_pred_train_lr_2, y_pred_test_lr_2)

Average expected loss: 2.639e+10
Average bias: 2.501e+10
Average variance: 1.376e+09
Training R^2 Score:  0.8643
Training RMSE: 133171
Testing R^2 Score:  0.8341
Testing RMSE: 158347


# Degree-3 Polynomial Features Model

In [43]:
poly_feat_3 = df.drop(['price', 'bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'zipcode'], axis=1)
poly_3 = PolynomialFeatures(degree=3, include_bias=False)
poly_data_3 = poly_3.fit_transform(poly_feat_3)
poly_columns_3 = poly_3.get_feature_names(poly_feat_3.columns)
df_poly_3 = pd.DataFrame(poly_data_3, columns=poly_columns_3)
X_poly_3 = pd.concat([df_poly_3, df_dum], axis=1)
X_poly_3.head()

,sqft_living,sqft_lot,waterfront,sqft_above,sqft_living15,sqft_lot15,sale_age,age,renovated,basement,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,1180.0,5650.0,0.0,1180.0,1340.0,5650.0,59.0,59.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1,2570.0,7242.0,0.0,2170.0,1690.0,7639.0,23.0,63.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,770.0,10000.0,0.0,770.0,2720.0,8062.0,82.0,82.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1960.0,5000.0,0.0,1050.0,1360.0,5000.0,49.0,49.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,1680.0,8080.0,0.0,1680.0,1800.0,7503.0,28.0,28.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_poly_3, y, random_state=42, test_size=0.2, shuffle=True)
lr_3 = LinearRegression().fit(X_train_3, y_train_3)
loss_3, bias_3, var_3 = bias_variance_decomp(lr_3, X_train_3.values, y_train_3.values, X_test_3.values, y_test_3.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_3)
print('Average bias: %.3e' % bias_3)
print('Average variance: %.3e' % var_3)
y_pred_train_lr_3 = lr_3.predict(X_train_3)
y_pred_test_lr_3 = lr_3.predict(X_test_3)
train_test_metrics(y_train_3, y_test_3, y_pred_train_lr_3, y_pred_test_lr_3)

Average expected loss: 1.853e+11
Average bias: 5.834e+10
Average variance: 1.270e+11
Training R^2 Score:  0.1103
Training RMSE: 340933
Testing R^2 Score:  0.5358
Testing RMSE: 264897


# Degree-4 Polynomal Features Model

In [45]:
poly_feat_4 = df.drop(['price', 'bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'zipcode'], axis=1)
poly_4 = PolynomialFeatures(degree=4, include_bias=False)
poly_data_4 = poly_4.fit_transform(poly_feat_4)
poly_columns_4 = poly_4.get_feature_names(poly_feat_4.columns)
df_poly_4 = pd.DataFrame(poly_data_4, columns=poly_columns_4)
X_poly_4 = pd.concat([df_poly_4, df_dum], axis=1)
X_poly_4.head()

,sqft_living,sqft_lot,waterfront,sqft_above,sqft_living15,sqft_lot15,sale_age,age,renovated,basement,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,1180.0,5650.0,0.0,1180.0,1340.0,5650.0,59.0,59.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1,2570.0,7242.0,0.0,2170.0,1690.0,7639.0,23.0,63.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,770.0,10000.0,0.0,770.0,2720.0,8062.0,82.0,82.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1960.0,5000.0,0.0,1050.0,1360.0,5000.0,49.0,49.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,1680.0,8080.0,0.0,1680.0,1800.0,7503.0,28.0,28.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_poly_4, y, random_state=42, test_size=0.2, shuffle=True)
lr_4 = LinearRegression().fit(X_train_4, y_train_4)
loss_4, bias_4, var_4 = bias_variance_decomp(lr_4, X_train_4.values, y_train_4.values, X_test_4.values, y_test_4.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_4)
print('Average bias: %.3e' % bias_4)
print('Average variance: %.3e' % var_4)
y_pred_train_lr_4 = lr_4.predict(X_train_4)
y_pred_test_lr_4 = lr_4.predict(X_test_4)
train_test_metrics(y_train_4, y_test_4, y_pred_train_lr_4, y_pred_test_lr_4)

Average expected loss: 3.484e+14
Average bias: 1.672e+13
Average variance: 3.317e+14
Training R^2 Score:  -237.9606
Training RMSE: 5587474
Testing R^2 Score:  -8555.4643
Testing RMSE: 35965791


# Metrics Dataframe

In [47]:
data = [['%.3e' % bias, '%.3e' % var, round(r2_score(y_train, y_pred_train_lr), 4), round(r2_score(y_test, y_pred_test_lr), 4), int(rmse(y_train, y_pred_train_lr)), int(rmse(y_test, y_pred_test_lr))],
        ['%.3e' % bias_2, '%.3e' % var_2, round(r2_score(y_train_2, y_pred_train_lr_2), 4), round(r2_score(y_test_2, y_pred_test_lr_2), 4), int(rmse(y_train_2, y_pred_train_lr_2)), int(rmse(y_test_2, y_pred_test_lr_2))],
        ['%.3e' % bias_3, '%.3e' % var_3, round(r2_score(y_train_3, y_pred_train_lr_3), 4), round(r2_score(y_test_3, y_pred_test_lr_3), 4), int(rmse(y_train_3, y_pred_train_lr_3)), int(rmse(y_test_3, y_pred_test_lr_3))],
        ['%.3e' % bias_4, '%.3e' % var_4, round(r2_score(y_train_4, y_pred_train_lr_4), 4), round(r2_score(y_test_4, y_pred_test_lr_4), 4), int(rmse(y_train_4, y_pred_train_lr_4)), int(rmse(y_test_4, y_pred_test_lr_4))]]

index = ["Baseline", "Poly-2", "Poly-3", "Poly-4"]
columns = ["Average Bias", "Average Variance", "Training R^2 Score", "Testing R^2 Score", "Training RMSE", "Testing RMSE"]

poly_feat_df = pd.DataFrame(data=data, index=index, columns=columns)
poly_feat_df

,Average Bias,Average Variance,Training R^2 Score,Testing R^2 Score,Training RMSE,Testing RMSE
Baseline,3.078e+10,1.676e+09,0.8328,0.8147,147801,167382
Poly-2,2.501e+10,1.376e+09,0.8643,0.8341,133171,158347
Poly-3,5.834e+10,1.270e+11,0.1103,0.5358,340933,264897
Poly-4,1.672e+13,3.317e+14,-237.9606,-8555.4643,5587474,35965791


In [49]:
poly_feat_df.to_csv("data/poly_feat_df.csv")

# Ridge Regression Model

In [27]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)
X_train_sc = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_sc = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [28]:
ridge = Ridge(alpha=1).fit(X_train_sc, y_train)
y_pred_train_ridge = ridge.predict(X_train_sc)
y_pred_test_ridge = ridge.predict(X_test_sc)
loss_5, bias_5, var_5 = bias_variance_decomp(ridge, X_train_sc.values, y_train.values, X_test_sc.values, y_test.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_5)
print('Average bias: %.3e' % bias_5)
print('Average variance: %.3e' % var_5)
train_test_metrics(y_train, y_test, y_pred_train_ridge, y_pred_test_ridge)

Average expected loss: 3.245e+10
Average bias: 3.079e+10
Average variance: 1.653e+09
Training R^2 Score:  0.8464
Training RMSE: 141645
Testing R^2 Score:  0.7729
Testing RMSE: 185283


# Lasso Regression Model

In [29]:
lasso = Lasso(alpha=1).fit(X_train_sc, y_train)
y_pred_train_lasso = lasso.predict(X_train_sc)
y_pred_test_lasso = lasso.predict(X_test_sc)
loss_6, bias_6, var_6 = bias_variance_decomp(lasso, X_train_sc.values, y_train.values, X_test_sc.values, y_test.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_6)
print('Average bias: %.3e' % bias_6)
print('Average variance: %.3e' % var_6)
train_test_metrics(y_train, y_test, y_pred_train_lasso, y_pred_test_lasso)

Average expected loss: 3.245e+10
Average bias: 3.083e+10
Average variance: 1.620e+09
Training R^2 Score:  0.8464
Training RMSE: 141647
Testing R^2 Score:  0.7729
Testing RMSE: 185285


# Decision Tree Regressor

In [30]:
dtr = DecisionTreeRegressor(random_state=42).fit(X_train, y_train)
y_pred_train_dtr = dtr.predict(X_train) 
y_pred_test_dtr = dtr.predict(X_test)
loss_7, bias_7, var_7 = bias_variance_decomp(dtr, X_train.values, y_train.values, X_test.values, y_test.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_7)
print('Average bias: %.3e' % bias_7)
print('Average variance: %.3e' % var_7)
train_test_metrics(y_train, y_test, y_pred_train_dtr, y_pred_test_dtr)


Average expected loss: 5.770e+10
Average bias: 2.922e+10
Average variance: 2.848e+10
Training R^2 Score:  0.9999
Training RMSE: 3466
Testing R^2 Score:  0.6293
Testing RMSE: 236718


# Random Forest Regressor

In [31]:
rf = RandomForestRegressor(n_estimators=100, 
                           max_features="auto", 
                           max_depth=100, 
                           min_samples_leaf=4, 
                           min_samples_split=10, 
                           random_state=1).fit(X_train, y_train)
y_pred_train_rf = rf.predict(X_train)
y_pred_test_rf= rf.predict(X_test)
loss_8, bias_8, var_8 = bias_variance_decomp(rf, X_train.values, y_train.values, X_test.values, y_test.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_8)
print('Average bias: %.3e' % bias_8)
print('Average variance: %.3e' % var_8)
train_test_metrics(y_train, y_test, y_pred_train_rf, y_pred_test_rf)

Average expected loss: 3.257e+10
Average bias: 2.927e+10
Average variance: 3.297e+09
Training R^2 Score:  0.9157
Training RMSE: 104960
Testing R^2 Score:  0.8044
Testing RMSE: 171969


# DataFrame of Metrics

In [36]:
data_2 = [['%.3e' % bias, '%.3e' % var, round(r2_score(y_train, y_pred_train_lr), 4), round(r2_score(y_test, y_pred_test_lr), 4), int(rmse(y_train, y_pred_train_lr)), int(rmse(y_test, y_pred_test_lr))],
        ['%.3e' % bias_5, '%.3e' % var_5, round(r2_score(y_train, y_pred_train_ridge), 4), round(r2_score(y_test, y_pred_test_ridge), 4), int(rmse(y_train, y_pred_train_ridge)), int(rmse(y_test, y_pred_test_ridge))],
        ['%.3e' % bias_6, '%.3e' % var_6, round(r2_score(y_train, y_pred_train_lasso), 4), round(r2_score(y_test, y_pred_test_lasso), 4), int(rmse(y_train, y_pred_train_lasso)), int(rmse(y_test, y_pred_test_lasso))],
        ['%.3e' % bias_7, '%.3e' % var_7, round(r2_score(y_train, y_pred_train_dtr), 4), round(r2_score(y_test, y_pred_test_dtr), 4), int(rmse(y_train, y_pred_train_dtr)), int(rmse(y_test, y_pred_test_dtr))],
        ['%.3e' % bias_8, '%.3e' % var_8, round(r2_score(y_train, y_pred_train_rf), 4), round(r2_score(y_test, y_pred_test_rf), 4), int(rmse(y_train, y_pred_train_rf)), int(rmse(y_test, y_pred_test_rf))]]

index_2 = ["Baseline", "Baseline w/Ridge", "Baseline w/Lasso", "Decision Tree Regressor", "Random Forest Regressor"]
columns = ["Average Bias", "Average Variance", "Training R^2 Score", "Testing R^2 Score", "Training RMSE", "Testing RMSE"]

models_df = pd.DataFrame(data=data_2, index=index_2, columns=columns)

In [37]:
models_df

,Average Bias,Average Variance,Training R^2 Score,Testing R^2 Score,Training RMSE,Testing RMSE
Baseline,3.078e+10,1.676e+09,0.8328,0.8147,147801,167382
Baseline w/Ridge,3.079e+10,1.653e+09,0.8464,0.7729,141645,185283
Baseline w/Lasso,3.083e+10,1.620e+09,0.8464,0.7729,141647,185285
Decision Tree Regressor,2.922e+10,2.848e+10,0.9999,0.6293,3466,236718
Random Forest Regressor,2.927e+10,3.297e+09,0.9157,0.8044,104960,171969


In [38]:
models_df.to_csv("data/models_df.csv")

In [50]:
ss = StandardScaler()
X_train_2_scaled = ss.fit_transform(X_train_2)
X_test_2_scaled = ss.transform(X_test_2)
X_train_2_sc = pd.DataFrame(X_train_2_scaled, columns=X_train_2.columns)
X_test_2_sc = pd.DataFrame(X_test_2_scaled, columns=X_test_2.columns)

In [51]:
ridge_2 = Ridge(alpha=1).fit(X_train_2_sc, y_train_2)
y_pred_train_2_ridge = ridge_2.predict(X_train_2_sc)
y_pred_test_2_ridge = ridge_2.predict(X_test_2_sc)
loss_9, bias_9, var_9 = bias_variance_decomp(ridge_2, X_train_2_sc.values, y_train_2.values, X_test_2_sc.values, y_test_2.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_9)
print('Average bias: %.3e' % bias_9)
print('Average variance: %.3e' % var_9)
train_test_metrics(y_train_2, y_test_2, y_pred_train_2_ridge, y_pred_test_2_ridge)

Average expected loss: 2.625e+10
Average bias: 2.489e+10
Average variance: 1.366e+09
Training R^2 Score:  0.8717
Training RMSE: 129447
Testing R^2 Score:  0.826
Testing RMSE: 162168


In [52]:
lasso = Lasso(alpha=1).fit(X_train_2_sc, y_train_2)
y_pred_train_2_lasso = lasso.predict(X_train_2_sc)
y_pred_test_2_lasso = lasso.predict(X_test_2_sc)
loss_10, bias_10, var_10 = bias_variance_decomp(lasso, X_train_2_sc.values, y_train_2.values, X_test_2_sc.values, y_test_2.values, loss='mse', random_seed=42)
print('Average expected loss: %.3e' % loss_10)
print('Average bias: %.3e' % bias_10)
print('Average variance: %.3e' % var_10)
train_test_metrics(y_train_2, y_test_2, y_pred_train_2_lasso, y_pred_test_2_lasso)

Average expected loss: 2.619e+10
Average bias: 2.484e+10
Average variance: 1.351e+09
Training R^2 Score:  0.8715
Training RMSE: 129558
Testing R^2 Score:  0.8266
Testing RMSE: 161891
